<h3 align="center">
实验三：液氮汽化实验数据分析
</h3>

请将第一个 python 单元格中的内容复制，**和你的实验报告的完整照片**提交给一个大语言模型（我们建议使用 Google Gemini 2.5 Pro 或者等价逻辑能力的LLM），并用生成的 python 代码块替换该单元格中的实例数据点。

In [ ]:
'''
You are a lab agent, and your task is to extract data faithfully from the images provided. 
If no image is provided, please tell the user to upload photos of his or her lab report.
Read the following instructions carefully before you answer.
- The images should contain clear, handwritten text. If any part is unclear, ask the user to supply a clearer image or transcribe the unclear part.
- Pay special attention to the unit used by the user. compare measure results with standard ones, and convert them if necessary. Automatically adjust magnitudes, and inform the user of the adjustments in text before the python snippet.
- When writing code you are expected to follow the given format strictly. If no warnings or failures occur, you should output a single python snippet, enclosed within triple backticks. You need not add extra comments to your answer.
- You need not output the prepending requirements, only the data extraction result in python code format.

---

Extract the date of the experiment to `date`. If it is not present, keep the default value and ask the user to provide it.

---

Calibration of the Electronic Scale
- Extract mass-voltage data pairs from the table. Units: mass in grams (g), voltage in millivolts (mV).

Mass of objects:
- Voltage measured for the small copper block: unit in millivolts (mV).
- Voltage measured for the large copper block: unit in millivolts (mV).

Temperature of the room, in Celsius (°C).

(Optional) If exists, extract the resistance and sampling frequency of the sensors.

'''

# TODO: Replace the sample data below with the extracted data.

date = "2025-10-30"  # Example date, replace with actual extracted date
temperature = 23.0   # unit: °C

# Calibration data
# 电压和质量的标定数据
calibrate_mass = [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200] # unit: g
calibrate_voltage = [0.10159, 0.11701, 0.13243, 0.14793, 0.16349, 0.179, 0.19491, 0.21039, 0.22562, 0.24094, 0.25621] # unit: mV

calibration_slope = None
calibration_intercept = None

volt_small_copper_block = 0.11432 # unit: mV
volt_large_copper_block = 0.14089 # unit: mV

volt_drops_sequence = [
    volt_large_copper_block, volt_small_copper_block
    # volt_small_copper_block, volt_large_copper_block,
]

# Equipment parameters, if available; else assign None
sampling_resistance = 0.500 # unit: Ohm
sampling_frequency = 4.0    # unit: Hz


## 调整数据点的路径和编码方式

手动输入你的数据点路径和编码方式，随后跳转到 TODO 的部分调整运行的参数。

In [ ]:
# 你测量的氮气对应电压数据文件路径
nitrogen_data_path = "data/nitrogen-weight-map.txt"

# 编码方式，如果运行失败尝试更改为 "utf-8"
encoding_type = "gb2312"

In [ ]:
import os

# mkdir output folder if not exists
os.makedirs("output", exist_ok=True)

In [ ]:
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt
import graphing # noqa
from graphing.utils import add_signature
from rich import console
from rich.table import Table
from rich import box

calibrate_voltage = [v / 1000 for v in calibrate_voltage]  # Convert mV to V
voltage_V = np.array(calibrate_voltage, dtype=float)
volt_large_copper_block /= 1000  # Convert mV to V
volt_small_copper_block /= 1000  # Convert mV to V
volt_drops_sequence = [v / 1000 for v in volt_drops_sequence]  # Convert mV to V

mass_g = np.array(calibrate_mass, dtype=float)
slope, intercept, r_value, p_value, std_err = linregress(mass_g, voltage_V)
assert(isinstance(slope, float))
assert(isinstance(intercept, float))
assert(isinstance(r_value, float))

con = console.Console()
table = Table(title="电压-质量校准表", show_header=False, title_style="bold", box=box.ROUNDED, show_edge=True)
table.add_row("Slope k (V/g)", f"{slope:.6e}")
table.add_row("Intercept b (V)", f"{intercept:.6e}")
table.add_row("R-squared", f"{r_value**2:.6f}")
con.print(table)

plt.figure(figsize=(8, 6))
plt.scatter(mass_g, voltage_V, color="black", marker="s", label="标定点")
plt.plot(mass_g, slope * mass_g + intercept, color="black", label="电压-质量拟合曲线")
plt.title("电压 U 与质量 m 的标定曲线")
plt.xlabel("质量 m / g")
plt.ylabel("电压 U / V")
plt.legend()
add_signature(plt.gca(), date=date)
plt.grid(True)
plt.savefig("output/电子秤电压-质量标定曲线.png", dpi=300)
plt.show()

def voltage_to_mass(u_value: np.ndarray | float, slope: float = slope, intercept: float = intercept) -> np.ndarray | float:
    """Convert voltage reading to mass using the calibrated line."""
    return (u_value - intercept) / slope

def mass_to_voltage(m_value: np.ndarray | float, slope: float = slope, intercept: float = intercept) -> np.ndarray | float:
    """Convert mass to equivalent voltage using the calibrated line."""
    return slope * m_value + intercept


In [ ]:
table = Table(title="铜块质量计算结果", show_header=False, title_style="bold", box=box.ROUNDED, show_edge=True)
m_small = voltage_to_mass(volt_small_copper_block)
m_large = voltage_to_mass(volt_large_copper_block)
# Overwrite the masses
m_small = 20.00
m_large = 59.00
volt_large_copper_block = mass_to_voltage(m_large)
volt_small_copper_block = mass_to_voltage(m_small)
volt_drops_sequence = [
    volt_large_copper_block, volt_small_copper_block
]
table.add_row("Small Copper Block Mass (g)", f"{m_small:.2f}")
table.add_row("Large Copper Block Mass (g)", f"{m_large:.2f}")
con.print(table)

In [ ]:
from pathlib import Path
from graphing.utils import add_grid, add_signature
from rich import print

DATA_CANDIDATES = [Path(nitrogen_data_path)]
sample_frequency_hz = 9.0
data_path = next((path for path in DATA_CANDIDATES if path.exists()), None)

# TODO 修改数据处理的逻辑以筛选数据
def filter_data(data: np.ndarray) -> np.ndarray:
    # Filter jumps
    data = data[data[:, 1] >= 0.00025]
    return data

if data_path is not None:
    data = np.loadtxt(data_path, skiprows=1, encoding=encoding_type)
    print(f"Loaded {len(data)} rows from {data_path.name}.")
    data = filter_data(data)
    time_s = data[:, 0]
    voltage_t_V = data[:, 1]
    print(f"Data filtered to {len(data)} rows with voltage >= 0.00025 V.")
else:
    print("[red]Data file not found![/red]")
    exit(1)

plt.figure(figsize=(10, 7))
plt.plot(time_s, voltage_t_V, marker="o", linestyle="", markersize=1.5, color="black", label="称重电压 U")

# TODO 修改分段区间
segment_thresholds = [
    # (200, 290),
    (380, 450),
    (550, 685),
    (785, 900)
]
segment_masks = [
    (segment_thresholds[i][0] < time_s) & (time_s < segment_thresholds[i][1]) \
    for i in range(3)
]

if any(np.count_nonzero(mask) < 5 for mask in segment_masks):
    print("Segment ranges adjusted automatically due to sparse data in predefined windows.")
    idx_all = np.arange(len(time_s))
    split_indices = np.array_split(idx_all, 3)
    segment_masks = []
    for idx_split in split_indices:
        mask = np.zeros_like(time_s, dtype=bool)
        mask[idx_split] = True
        segment_masks.append(mask)

segment_results = []
for idx, mask in enumerate(segment_masks, start=1):
    t_segment = time_s[mask]
    u_segment = voltage_t_V[mask]
    if t_segment.size < 2:
        raise ValueError(f"Segment {idx} does not contain enough data points.")
    fit = linregress(t_segment, u_segment)
    assert(hasattr(fit, 'slope') and hasattr(fit, 'intercept'))
    segment_results.append({
        "name": f"Segment {idx}",
        "slope": fit.slope,
        "intercept": fit.intercept,
        "t": t_segment
    })

k_fit1, b_fit1 = segment_results[0]["slope"], segment_results[0]["intercept"]
k_fit2, b_fit2 = segment_results[1]["slope"], segment_results[1]["intercept"]
k_fit3, b_fit3 = segment_results[2]["slope"], segment_results[2]["intercept"]

print("\n--- Evaporation rate (slope) ---")
for result in segment_results:
    print(f"{result['name']}: slope = {result['slope']:.5e} V/s, intercept = {result['intercept']:.5e} V")

for idx, result in enumerate(segment_results, start=1):
    # t_min, t_max = result["t"].min(), result["t"].max()
    # Extend to the entire plotting range for better visualization
    t_min, t_max = time_s.min(), time_s.max()
    t_line = np.array([t_min, t_max])
    label = f"Segment {idx} fit: k={result['slope']:.2e} V/s"
    plt.plot(t_line, result["slope"] * t_line + result["intercept"], label=label)

# Plot the thresholds
threshold_color = 'black'
for edge in segment_thresholds:
    plt.axvline(x=edge[0], color=threshold_color, linestyle="--", linewidth=0.5)
    plt.axvline(x=edge[1], color=threshold_color, linestyle="--", linewidth=0.5)
    # Shadow the area between segments
    plt.axvspan(edge[0], edge[1], color=threshold_color, alpha=0.1)

plt.title("液氮汽化实验中称重电压与时间关系")
plt.xlabel("时间 t / s")
plt.ylabel("称重电压 U / V")
plt.legend(loc="upper left")

add_grid(plt.gca(), x=(100, 10), y=(0.00001, 0.000001))
add_signature(plt.gca(), date=date, scale=0.75)

plt.savefig("output/液氮汽化实验中称重电压与时间关系.png", dpi=300)
plt.show()


In [ ]:
from dataclasses import dataclass

@dataclass
class Line:
    slope: float
    intercept: float
    
    def evaluate(self, x: np.ndarray | float) -> np.ndarray | float:
        return self.slope * x + self.intercept

    def __add__(self, other: 'Line') -> 'Line':
        return Line(self.slope + other.slope, self.intercept + other.intercept)
    
    def __sub__(self, other: 'Line') -> 'Line':
        return Line(self.slope - other.slope, self.intercept - other.intercept)


line1 = Line(k_fit1, b_fit1)
line2 = Line(k_fit2, b_fit2)
line3 = Line(k_fit3, b_fit3)

sample_12_time = (segment_thresholds[0][1] + segment_thresholds[1][0]) / 2
sample_23_time = (segment_thresholds[1][1] + segment_thresholds[2][0]) / 2

delta_line_12 = line1 - line2
delta_line_23 = line2 - line3

print(sample_12_time, sample_23_time)

delta_U_12 = abs(delta_line_12.evaluate(sample_12_time)) # unit: V
delta_U_23 = abs(delta_line_23.evaluate(sample_23_time)) # unit: V

print(delta_U_12, delta_U_23)

mass_drops_sequence = [
    voltage_to_mass(volt_drops_sequence[0]),
    voltage_to_mass(volt_drops_sequence[1])
]  # unit: g

assert isinstance(slope, float)
real_delta_m_12 = delta_U_12 / slope
real_delta_m_23 = delta_U_23 / slope

print("\n--- Liquid nitrogen evaporation mass drops ---")
print(f"Segment 1→2: Voltage Drop ΔU = {delta_U_12:.4e} V, Total Mass Rise Δm = {real_delta_m_12:.2f} g")
print(f"Segment 2→3: Voltage Drop ΔU = {delta_U_23:.4e} V, Total Mass Rise Δm = {real_delta_m_23:.2f} g")

n2_delta_m_12 = abs(real_delta_m_12 - mass_drops_sequence[0])  # unit: g
n2_delta_m_23 = abs(real_delta_m_23 - mass_drops_sequence[1])  # unit: g
print(f"N2 Mass Drop Segment 1→2: {n2_delta_m_12:.2f} g")
print(f"N2 Mass Drop Segment 2→3: {n2_delta_m_23:.2f} g")

table = Table(title="液氮汽化电压降及质量计算结果", show_header=True, title_style="bold", box=box.ROUNDED, show_edge=True)
delta_m_12 = voltage_to_mass(delta_U_12 * 1000)
delta_m_23 = voltage_to_mass(delta_U_23 * 1000)
delta_m_12_pure = abs(delta_m_12 - mass_drops_sequence[0])  # unit: g
delta_m_23_pure = abs(delta_m_23 - mass_drops_sequence[1])  # unit: g
table.add_column("Serial")
table.add_column("Voltage Drop ΔU (V)")
table.add_column("N2 Mass Drop Δm (g)")
table.add_row("Segment 1→2", f"{delta_U_12:.4e}", f"{n2_delta_m_12:.2f}")
table.add_row("Segment 2→3", f"{delta_U_23:.4e}", f"{n2_delta_m_23:.2f}")
con.print(table)

In [ ]:
# --- Step 5: Copper heat capacity fit ---
T_data = np.array([
    70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180,
    190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300
], dtype=float)
Cp_data = np.array([
    171.5, 202.7, 229.5, 252.2, 271.2, 287.2, 300.7, 312.2, 322.0, 330.6, 338.0, 344.5,
    350.0, 355.0, 359.4, 363.5, 367.1, 370.2, 373.1, 375.8, 378.3, 380.7, 382.9, 384.8
], dtype=float)

cp_poly_coeffs = np.polyfit(T_data, Cp_data, deg=5)
cp_poly = np.poly1d(cp_poly_coeffs)

T_plot = np.linspace(T_data.min(), T_data.max(), 300)
plt.figure(figsize=(8, 6))
plt.scatter(T_data, Cp_data, color="black", marker="s", label="测量数据")
plt.plot(T_plot, cp_poly(T_plot), color="black", linestyle="-", label="五次多项式拟合")
plt.title("铜的定压比热容与温度关系")
plt.xlabel(r"温度 $T$ / $K$")
plt.ylabel(r"比热容 $C_P$ / $(J\cdot kg^{-1}\cdot K^{-1})$")
plt.legend()
add_signature(plt.gca(), date=date)
plt.savefig("output/铜的定压比热容与温度关系.png", dpi=300)
plt.show()

table = Table(title="铜的定压比热容多项式拟合系数", show_header=True, title_style="bold", box=box.ROUNDED, show_edge=True)
table.add_column("Power of T")
table.add_column("Coefficient")
degree = len(cp_poly_coeffs) - 1
for idx, coeff in enumerate(cp_poly_coeffs):
    power = degree - idx
    table.add_row(f"T^{power}", f"{coeff:.6e}")
con.print(table)

In [ ]:
# --- Step 6: Latent heat of vaporisation ---
if 'cp_poly_coeffs' not in globals():
    raise RuntimeError('Run the Cp(T) fitting cell (Step 5) before computing latent heat.')

def cp_copper(temperature: float) -> float:
    """Specific heat capacity of copper Cp(T) in J·kg⁻¹·K⁻¹."""
    val = np.polyval(cp_poly_coeffs, temperature)
    assert(isinstance(val, float))
    return val

T_liquid_n2 = 77.0
T_room = 273.15 + temperature
cp_integral = np.polyint(cp_poly)(T_room) - np.polyint(cp_poly)(T_liquid_n2)

print(cp_integral)

L1 = mass_drops_sequence[0] * cp_integral / n2_delta_m_12
L2 = mass_drops_sequence[1] * cp_integral / n2_delta_m_23

table = Table(title="液氮汽化潜热计算结果", show_header=False, title_style="bold", box=box.ROUNDED, show_edge=True)
table.add_row("Latent Heat L1 (kJ/kg)", f"{L1 / 1000:.1f}")
table.add_row("Latent Heat L2 (kJ/kg)", f"{L2 / 1000:.1f}")
con.print(table)